## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [2]:
# Get today's date
today = datetime.now()

today = today - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

Feb. 10th Feb. 17th


In [3]:
filename = glob.glob('redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [4]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [5]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [6]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [7]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,January-22-2025,Condo/Co-op,10302 Myrtlewood Cir W,Palm Beach Gardens,FL,33418.0,340000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach-Gardens/1...,Beaches MLS,RX-11044786,N,Y,26.873428,-80.106892


In [8]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [11]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [14]:
df2 = df[(df['SOLD DATE'] > '2025-02-10') & (df['SOLD DATE'] < '2025-02-17')]

In [15]:
df2['SOLD DATE'].value_counts()

SOLD DATE
2025-02-14    41
2025-02-11    18
2025-02-13    15
2025-02-12    12
2025-02-15     1
Name: count, dtype: int64

In [16]:
df2 = df2.sort_values(by='PRICE',ascending=False)

In [69]:
# df2 = df2.drop(index=64)

In [17]:
df_top_ten = df2.head(10)

In [18]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
78,PAST SALE,2025-02-12,Condo/Co-op,200 SE Mizner Blvd #203,Boca Raton,FL,33432.0,3190000.0,2.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/200-SE-Mi...,Beaches MLS,RX-11041137,N,Y,26.347715,-80.083263
148,PAST SALE,2025-02-14,Condo/Co-op,300 SE 5th Ave #7120,Boca Raton,FL,33432.0,1800000.0,3.0,4.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/300-SE-5t...,Beaches MLS,RX-11023565,N,Y,26.346087,-80.076579
227,PAST SALE,2025-02-14,Condo/Co-op,4740 S Ocean Blvd #1610,Highland Beach,FL,33487.0,1500000.0,3.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/4740-...,Beaches MLS,RX-10899388,N,Y,26.393421,-80.069121
159,PAST SALE,2025-02-14,Condo/Co-op,4161 S Us Highway 1 Unit K4,Jupiter,FL,33477.0,1400000.0,4.0,3.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/4161-S-US-Hi...,Beaches MLS,RX-11025987,N,Y,26.894166,-80.058991
110,PAST SALE,2025-02-12,Condo/Co-op,500 S Ocean Blvd #1103,Boca Raton,FL,33432.0,1370000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/500-S-Oce...,Beaches MLS,RX-11034349,N,Y,26.345068,-80.071816
179,PAST SALE,2025-02-14,Condo/Co-op,2800 S Ocean Blvd Unit 10-K,Boca Raton,FL,33432.0,1160000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/2800-S-Oc...,Beaches MLS,RX-11008547,N,Y,26.323476,-80.075019
131,PAST SALE,2025-02-14,Condo/Co-op,400 Seasage Dr #204,Delray Beach,FL,33483.0,1075000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/400-Sea...,Beaches MLS,RX-11034650,N,Y,26.454271,-80.063637
99,PAST SALE,2025-02-12,Condo/Co-op,99 Tresana Blvd #5,Jupiter,FL,33478.0,885000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/99-Tresana-B...,Beaches MLS,RX-11041325,N,Y,26.925797,-80.159584
211,PAST SALE,2025-02-14,Condo/Co-op,2000 S Ocean Blvd #707,Delray Beach,FL,33483.0,850000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/2000-S-...,Beaches MLS,RX-10993954,N,Y,26.436069,-80.063751
178,PAST SALE,2025-02-13,Condo/Co-op,318 SE 7th Ave #3,Delray Beach,FL,33483.0,810000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/318-SE-...,Beaches MLS,RX-11017285,N,Y,26.455484,-80.066176


In [12]:
# df_top_ten = df2.head(10)

## Collect Agent Information

In [19]:
def process_response(url):
    try:
        
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

        querystring = {"url":url}

        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)

#         print(response.status_code)
#         print(url)
        time.sleep(0.2)

        response.raise_for_status()
        data = response.json()

        # Initialize default values for listing and buyer agents data
        list_agent_name_1 = None
        list_broker_name_1 = None
        list_agent_name_2 = None
        list_broker_name_2 = None
        
        buy_agent_name_1 = None
        buy_broker_name_1 = None
        buy_agent_name_2 = None
        buy_broker_name_2 = None
        
        # Extract listing agents data if it exists
        listing_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('listingAgents', [])
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')
            
        # Extract listing agents data if it exists
        buying_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('buyingAgents', [])
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}).get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}).get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')


        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }

In [20]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [21]:
response_df = pd.DataFrame(response_list)

In [22]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [23]:
df_top_ten = merged_df

In [24]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2
0,PAST SALE,2025-02-12,Condo/Co-op,200 SE Mizner Blvd #203,Boca Raton,FL,33432.0,3190000.0,2.0,3.5,...,-80.083263,https://www.redfin.com/FL/Boca-Raton/200-SE-Mi...,Senada Adzem,Douglas Elliman,Denver Bright,"Compass Florida, LLC",Aaron Buchbinder,Compass Florida LLC,Steven Seigel,The Agency Florida LLC
1,PAST SALE,2025-02-14,Condo/Co-op,300 SE 5th Ave #7120,Boca Raton,FL,33432.0,1800000.0,3.0,4.5,...,-80.076579,https://www.redfin.com/FL/Boca-Raton/300-SE-5t...,Sara Brown,Coldwell Banker/BR,None,None,Adam Levitt,The Corcoran Group,None,None
2,PAST SALE,2025-02-14,Condo/Co-op,4740 S Ocean Blvd #1610,Highland Beach,FL,33487.0,1500000.0,3.0,2.5,...,-80.069121,https://www.redfin.com/FL/Highland-Beach/4740-...,Gina Finsilver,Compass Florida LLC,None,None,Carol Rochon,"United Realty Group, Inc",None,None
3,PAST SALE,2025-02-14,Condo/Co-op,4161 S Us Highway 1 Unit K4,Jupiter,FL,33477.0,1400000.0,4.0,3.0,...,-80.058991,https://www.redfin.com/FL/Jupiter/4161-S-US-Hi...,Andrew Russo,Illustrated Properties LLC (Jupiter),Milla Russo,Illustrated Properties LLC (Jupiter),Mary Ellen Kavanewsky,"NV Realty Group, LLC",None,None
4,PAST SALE,2025-02-12,Condo/Co-op,500 S Ocean Blvd #1103,Boca Raton,FL,33432.0,1370000.0,2.0,2.0,...,-80.071816,https://www.redfin.com/FL/Boca-Raton/500-S-Oce...,Ana Londono,Lang Realty/BR,None,None,Kun Li,Rainbow Investment Realty Inc,None,None
5,PAST SALE,2025-02-14,Condo/Co-op,2800 S Ocean Blvd Unit 10-K,Boca Raton,FL,33432.0,1160000.0,2.0,2.0,...,-80.075019,https://www.redfin.com/FL/Boca-Raton/2800-S-Oc...,Richard Dusik,Florida Beach Realty,None,None,Richard Dusik,Florida Beach Realty,None,None
6,PAST SALE,2025-02-14,Condo/Co-op,400 Seasage Dr #204,Delray Beach,FL,33483.0,1075000.0,2.0,2.0,...,-80.063637,https://www.redfin.com/FL/Delray-Beach/400-Sea...,Scott Porter,John P. O'Grady LLC,None,None,Charles Lynn,One Sotheby's International Realty,None,None
7,PAST SALE,2025-02-12,Condo/Co-op,99 Tresana Blvd #5,Jupiter,FL,33478.0,885000.0,2.0,2.0,...,-80.159584,https://www.redfin.com/FL/Jupiter/99-Tresana-B...,Sam Elias,The Agency Florida LLC,Rosemary Elias,The Agency Florida LLC,Diane Gray,"NV Realty Group, LLC",None,None
8,PAST SALE,2025-02-14,Condo/Co-op,2000 S Ocean Blvd #707,Delray Beach,FL,33483.0,850000.0,2.0,2.0,...,-80.063751,https://www.redfin.com/FL/Delray-Beach/2000-S-...,Aixa Diaz-Monaco,Coldwell Banker Realty /Delray Beach,None,None,Lisa Nanfra,Compass Florida LLC,None,None
9,PAST SALE,2025-02-13,Condo/Co-op,318 SE 7th Ave #3,Delray Beach,FL,33483.0,810000.0,2.0,2.0,...,-80.066176,https://www.redfin.com/FL/Delray-Beach/318-SE-...,William Schloesser,Houseworks Real Estate,Carol Fredericks,Houseworks Real Estate,Carol Fredericks,Houseworks Real Estate,William Schloesser,Houseworks Real Estate


## Current Week's Values

In [25]:
# Number of condos sold
current_week_condo_sales = len(df2)
print(len(df2))

87


In [26]:
# Average sale price
average_sale_price = round(df2['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$382,990


In [27]:
# Average PSF
average_price_psf = round(df2['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$269


In [28]:
# Total sales dollar volume
current_week_condo_sales_total = df2['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$33,320,130


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [22]:
# # Define the date range
# start_date = '2024-12-23'
# end_date = '2024-12-29'

# # Filter the DataFrame between two dates (inclusive)
# old_week = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [21]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [29]:
print('Input county name:')
county = input()
print('Input Previous Week Condo Sales Total (number units sold):')
previous_week_condo_sales = input()
print('Input Previous Week Condo Average Sales Price:')
previous_week_average_sales_price = input()
print('Input Previous Week Condo Average PSF:')
previous_week_average_psf = input()
print('Input Previous Week Condo Sales total (ex: 198_000_000)')
previous_week_condo_sales_total = input()

## Clean Data

Input county name:
Palm Beach County
Input Previous Week Condo Sales Total (number units sold):
70
Input Previous Week Condo Average Sales Price:
750000
Input Previous Week Condo Average PSF:
319
Input Previous Week Condo Sales total (ex: 198_000_000)
61_300_000


In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [30]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [31]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [32]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [33]:
pd.set_option('display.max_columns',None)

In [34]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-02-12,Condo/Co-op,200 SE Mizner Blvd #203,Boca Raton,FL,33432.0,3190000.0,2.0,3.5,Alina Residences,2713.0,NaN,2020.0,NaN,1176.0,3329.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/200-SE-Mi...,Beaches MLS,RX-11041137,N,Y,26.347715,-80.083263,https://www.redfin.com/FL/Boca-Raton/200-SE-Mi...,Senada Adzem,Douglas Elliman,Denver Bright,"Compass Florida, LLC",Aaron Buchbinder,Compass Florida LLC,Steven Seigel,The Agency Florida LLC,1,orange


In [35]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [36]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-02-12,Condo/Co-op,200 SE Mizner Blvd #203,Boca Raton,FL,33432.0,3190000.0,2.0,3.5,Alina Residences,2713.0,NaN,2020.0,NaN,1176.0,3329.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/200-SE-Mi...,Beaches MLS,RX-11041137,N,Y,26.347715,-80.083263,https://www.redfin.com/FL/Boca-Raton/200-SE-Mi...,Senada Adzem,Douglas Elliman,Denver Bright,"Compass Florida, LLC",Aaron Buchbinder,Compass Florida LLC,Steven Seigel,The Agency Florida LLC,1,orange
1,PAST SALE,2025-02-14,Condo/Co-op,300 SE 5th Ave #7120,Boca Raton,FL,33432.0,1800000.0,3.0,4.5,Mizner Tower,2465.0,NaN,1989.0,NaN,730.0,3485.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/300-SE-5t...,Beaches MLS,RX-11023565,N,Y,26.346087,-80.076579,https://www.redfin.com/FL/Boca-Raton/300-SE-5t...,Sara Brown,Coldwell Banker/BR,None,None,Adam Levitt,The Corcoran Group,None,None,2,blue
2,PAST SALE,2025-02-14,Condo/Co-op,4740 S Ocean Blvd #1610,Highland Beach,FL,33487.0,1500000.0,3.0,2.5,Braemar,2070.0,NaN,1982.0,NaN,725.0,2144.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/4740-...,Beaches MLS,RX-10899388,N,Y,26.393421,-80.069121,https://www.redfin.com/FL/Highland-Beach/4740-...,Gina Finsilver,Compass Florida LLC,None,None,Carol Rochon,"United Realty Group, Inc",None,None,3,blue
3,PAST SALE,2025-02-14,Condo/Co-op,4161 S Us Highway 1 Unit K4,Jupiter,FL,33477.0,1400000.0,4.0,3.0,Beachcomber Condo,2370.0,NaN,1984.0,NaN,591.0,1595.0,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/4161-S-US-Hi...,Beaches MLS,RX-11025987,N,Y,26.894166,-80.058991,https://www.redfin.com/FL/Jupiter/4161-S-US-Hi...,Andrew Russo,Illustrated Properties LLC (Jupiter),Milla Russo,Illustrated Properties LLC (Jupiter),Mary Ellen Kavanewsky,"NV Realty Group, LLC",None,None,4,blue
4,PAST SALE,2025-02-12,Condo/Co-op,500 S Ocean Blvd #1103,Boca Raton,FL,33432.0,1370000.0,2.0,2.0,Chalfonte Condo,1446.0,NaN,1974.0,NaN,947.0,1525.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/500-S-Oce...,Beaches MLS,RX-11034349,N,Y,26.345068,-80.071816,https://www.redfin.com/FL/Boca-Raton/500-S-Oce...,Ana Londono,Lang Realty/BR,None,None,Kun Li,Rainbow Investment Realty Inc,None,None,5,blue
5,PAST SALE,2025-02-14,Condo/Co-op,2800 S Ocean Blvd Unit 10-K,Boca Raton,FL,33432.0,1160000.0,2.0,2.0,Ocean Towers Condominium,1423.0,NaN,1976.0,NaN,815.0,1262.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/2800-S-Oc...,Beaches MLS,RX-11008547,N,Y,26.323476,-80.075019,https://www.redfin.com/FL/Boca-Raton/2800-S-Oc...,Richard Dusik,Florida Beach Realty,None,None,Richard Dusik,Florida Beach Realty,None,None,6,blue
6,PAST SALE,2025-02-14,Condo/Co-op,400 Seasage Dr #204,Delray Beach,FL,33483.0,1075000.0,2.0,2.0,Seagate Manor Condo,1760.0,NaN,1969.0,NaN,611.0,1504.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/400-Sea...,Beaches MLS,RX-11034650,N,Y,26.454271,-80.063637,https://www.redfin.com/FL/Delray-Beach/400-Sea...,Scott Porter,John P. O'Grady LLC,None,None,Charles Lynn,One Sotheby's International Realty,None,None,7,blue
7,PAST SALE,2025-02-12,Condo/Co-op,99 Tresana Blvd #5,Jupiter,FL,33478.0,885000.0,2.0,2.0,Jupiter Country Club,2124.0,NaN,2013.0,NaN,417.0,1293.0,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/99-Tresana-B...,Beaches MLS,RX-11041325,N,Y,26.925797,-80.159584,https://www.redfin.com/FL/Jupiter/99-Tresana-B...,Sam Elias,The Agency Florida LLC,Rosemary Elias,The Agency Florida LLC,Diane Gray,"NV Realty Group, LLC",None,None,8,blue
8,PAST SALE,2025-02-14,Condo/Co-op,2000 S Ocean Blvd #707,Delray Beach,FL,33483.0,850000.0,2.0,2.0,Delray Beach Club Apts Cond

In [37]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent {county} County Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} — {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [38]:
m.save('index.html')

## Data snagger

In [40]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [41]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-02-12,Condo/Co-op,200 SE Mizner Blvd #203,Boca Raton,FL,33432.0,3190000.0,2.0,3.5,Alina Residences,2713.0,NaN,2020.0,NaN,1176.0,3329.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/200-SE-Mi...,Beaches MLS,RX-11041137,N,Y,26.347715,-80.083263,https://www.redfin.com/FL/Boca-Raton/200-SE-Mi...,Senada Adzem,Douglas Elliman,Denver Bright,"Compass Florida, LLC",Aaron Buchbinder,Compass Florida LLC,Steven Seigel,The Agency Florida LLC,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [42]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/PalmBeach_County_condo_sales_week_ending_02182024


# CREATE TEMPLATE 

In [43]:
muni_set = set(df_top_ten['CITY'])

In [44]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [45]:
df_top_ten.reset_index(inplace=True,drop=True)

In [46]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [47]:
top_sale

'Alina Residences at 200 SE Mizner Blvd #203 in Boca Raton'

In [48]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [49]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [50]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-02-12,Condo/Co-op,200 SE Mizner Blvd #203,Boca Raton,FL,33432.0,3190000.0,2.0,3.5,Alina Residences,2713.0,NaN,2020.0,NaN,1176.0,3329.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/200-SE-Mi...,Beaches MLS,RX-11041137,N,Y,26.347715,-80.083263,https://www.redfin.com/FL/Boca-Raton/200-SE-Mi...,Senada Adzem,Douglas Elliman,Denver Bright,"Compass Florida, LLC",Aaron Buchbinder,Compass Florida LLC,Steven Seigel,The Agency Florida LLC,1,orange,Alina Residences at 200 SE Mizner Blvd #203 in...
1,PAST SALE,2025-02-14,Condo/Co-op,300 SE 5th Ave #7120,Boca Raton,FL,33432.0,1800000.0,3.0,4.5,Mizner Tower,2465.0,NaN,1989.0,NaN,730.0,3485.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/300-SE-5t...,Beaches MLS,RX-11023565,N,Y,26.346087,-80.076579,https://www.redfin.com/FL/Boca-Raton/300-SE-5t...,Sara Brown,Coldwell Banker/BR,None,None,Adam Levitt,The Corcoran Group,None,None,2,blue,Mizner Tower at 300 SE 5th Ave #7120 in Boca R...
2,PAST SALE,2025-02-14,Condo/Co-op,4740 S Ocean Blvd #1610,Highland Beach,FL,33487.0,1500000.0,3.0,2.5,Braemar,2070.0,NaN,1982.0,NaN,725.0,2144.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/4740-...,Beaches MLS,RX-10899388,N,Y,26.393421,-80.069121,https://www.redfin.com/FL/Highland-Beach/4740-...,Gina Finsilver,Compass Florida LLC,None,None,Carol Rochon,"United Realty Group, Inc",None,None,3,blue,Braemar at 4740 S Ocean Blvd #1610 in Highland...
3,PAST SALE,2025-02-14,Condo/Co-op,4161 S Us Highway 1 Unit K4,Jupiter,FL,33477.0,1400000.0,4.0,3.0,Beachcomber Condo,2370.0,NaN,1984.0,NaN,591.0,1595.0,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/4161-S-US-Hi...,Beaches MLS,RX-11025987,N,Y,26.894166,-80.058991,https://www.redfin.com/FL/Jupiter/4161-S-US-Hi...,Andrew Russo,Illustrated Properties LLC (Jupiter),Milla Russo,Illustrated Properties LLC (Jupiter),Mary Ellen Kavanewsky,"NV Realty Group, LLC",None,None,4,blue,Beachcomber Condo at 4161 S Us Highway 1 Unit ...
4,PAST SALE,2025-02-12,Condo/Co-op,500 S Ocean Blvd #1103,Boca Raton,FL,33432.0,1370000.0,2.0,2.0,Chalfonte Condo,1446.0,NaN,1974.0,NaN,947.0,1525.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/500-S-Oce...,Beaches MLS,RX-11034349,N,Y,26.345068,-80.071816,https://www.redfin.com/FL/Boca-Raton/500-S-Oce...,Ana Londono,Lang Realty/BR,None,None,Kun Li,Rainbow Investment Realty Inc,None,None,5,blue,Chalfonte Condo at 500 S Ocean Blvd #1103 in B...
5,PAST SALE,2025-02-14,Condo/Co-op,2800 S Ocean Blvd Unit 10-K,Boca Raton,FL,33432.0,1160000.0,2.0,2.0,Ocean Towers Condominium,1423.0,NaN,1976.0,NaN,815.0,1262.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/2800-S-Oc...,Beaches MLS,RX-11008547,N,Y,26.323476,-80.075019,https://www.redfin.com/FL/Boca-Raton/2800-S-Oc...,Richard Dusik,Florida Beach Realty,None,None,Richard Dusik,Florida Beach Realty,None,None,6,blue,Ocean Towers Condominium at 2800 S Ocean Blvd ...
6,PAST SALE,2025-02-14,Condo/Co-op,400 Seasage Dr #204,Delray Beach,FL,33483.0,1075000.0,2.0,2.0,Seagate Manor Condo,1760.0,NaN,1969.0,NaN,611.0,1504.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/400-Sea...,Beaches MLS,RX-11034650,N,Y,26.454271,-80.063637,https://www.redfin.com/FL/Delray-Beach/400-Sea...,Scott Porter,John P. O'Grady LLC,None,None,Charles Lynn,One Sotheby's International Realty,None,None,7,blue,Seagate Manor Condo at 400 Seasage Dr #204 in ...
7,PAST SALE,2025-02-12,Condo/Co-op,99 Tresana Blvd #5,Jupiter,FL,33478.0,885000.0,2.0,2.0,Jupiter Country Club,2124.0,NaN,2013.0,NaN,417.0,1293.0,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/99-

In [51]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f} to ${df_top_ten.at[0,'PRICE']:,.0f}
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Kate Hinsche
\033[1mRESEARCH:\033[0m Adam Farence
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Image of the top sale

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: Alina Residences closing tops Palm Beach County weekly condo sales 
DEK: Top 10 sales ranged from $810,000 to $3,190,000
FEATURED HED:
SEO HED: Palm Beach County County Weekly Condo Report 
SEO DESCRIPTION: Palm Beach County County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Palm Beach County condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Palm Beach County, weekly condo sales, Boca Raton, Jupiter, Highland Beach, Delray Beach

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Palm Beach County County’s NEWS PEG HERE.

Brokers closed 87 condo sales totaling $33,320,130 million from Feb. 10th to Feb. 17th. The previous week, brokers closed 70 condo sales totaling $61,300,000.

Last week’s units sold for an average of $382,990, lower than the $750,000 average pric

In [52]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [53]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [54]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [55]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-02-12,Condo/Co-op,200 SE Mizner Blvd #203,Boca Raton,FL,33432.0,3190000.0,2.0,3.5,Alina Residences,2713.0,NaN,2020.0,NaN,1176.0,3329.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/200-SE-Mi...,Beaches MLS,RX-11041137,N,Y,26.347715,-80.083263,https://www.redfin.com/FL/Boca-Raton/200-SE-Mi...,Senada Adzem,Douglas Elliman,Denver Bright,"Compass Florida, LLC",Aaron Buchbinder,Compass Florida LLC,Steven Seigel,The Agency Florida LLC,1,orange,Alina Residences at 200 SE Mizner Blvd #203 in...
1,PAST SALE,2025-02-14,Condo/Co-op,300 SE 5th Ave #7120,Boca Raton,FL,33432.0,1800000.0,3.0,4.5,Mizner Tower,2465.0,NaN,1989.0,NaN,730.0,3485.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/300-SE-5t...,Beaches MLS,RX-11023565,N,Y,26.346087,-80.076579,https://www.redfin.com/FL/Boca-Raton/300-SE-5t...,Sara Brown,Coldwell Banker/BR,None,None,Adam Levitt,The Corcoran Group,None,None,2,blue,Mizner Tower at 300 SE 5th Ave #7120 in Boca R...
2,PAST SALE,2025-02-14,Condo/Co-op,4740 S Ocean Blvd #1610,Highland Beach,FL,33487.0,1500000.0,3.0,2.5,Braemar,2070.0,NaN,1982.0,NaN,725.0,2144.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/4740-...,Beaches MLS,RX-10899388,N,Y,26.393421,-80.069121,https://www.redfin.com/FL/Highland-Beach/4740-...,Gina Finsilver,Compass Florida LLC,None,None,Carol Rochon,"United Realty Group, Inc",None,None,3,blue,Braemar at 4740 S Ocean Blvd #1610 in Highland...
3,PAST SALE,2025-02-14,Condo/Co-op,4161 S Us Highway 1 Unit K4,Jupiter,FL,33477.0,1400000.0,4.0,3.0,Beachcomber Condo,2370.0,NaN,1984.0,NaN,591.0,1595.0,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/4161-S-US-Hi...,Beaches MLS,RX-11025987,N,Y,26.894166,-80.058991,https://www.redfin.com/FL/Jupiter/4161-S-US-Hi...,Andrew Russo,Illustrated Properties LLC (Jupiter),Milla Russo,Illustrated Properties LLC (Jupiter),Mary Ellen Kavanewsky,"NV Realty Group, LLC",None,None,4,blue,Beachcomber Condo at 4161 S Us Highway 1 Unit ...
4,PAST SALE,2025-02-12,Condo/Co-op,500 S Ocean Blvd #1103,Boca Raton,FL,33432.0,1370000.0,2.0,2.0,Chalfonte Condo,1446.0,NaN,1974.0,NaN,947.0,1525.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/500-S-Oce...,Beaches MLS,RX-11034349,N,Y,26.345068,-80.071816,https://www.redfin.com/FL/Boca-Raton/500-S-Oce...,Ana Londono,Lang Realty/BR,None,None,Kun Li,Rainbow Investment Realty Inc,None,None,5,blue,Chalfonte Condo at 500 S Ocean Blvd #1103 in B...
5,PAST SALE,2025-02-14,Condo/Co-op,2800 S Ocean Blvd Unit 10-K,Boca Raton,FL,33432.0,1160000.0,2.0,2.0,Ocean Towers Condominium,1423.0,NaN,1976.0,NaN,815.0,1262.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/2800-S-Oc...,Beaches MLS,RX-11008547,N,Y,26.323476,-80.075019,https://www.redfin.com/FL/Boca-Raton/2800-S-Oc...,Richard Dusik,Florida Beach Realty,None,None,Richard Dusik,Florida Beach Realty,None,None,6,blue,Ocean Towers Condominium at 2800 S Ocean Blvd ...
6,PAST SALE,2025-02-14,Condo/Co-op,400 Seasage Dr #204,Delray Beach,FL,33483.0,1075000.0,2.0,2.0,Seagate Manor Condo,1760.0,NaN,1969.0,NaN,611.0,1504.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/400-Sea...,Beaches MLS,RX-11034650,N,Y,26.454271,-80.063637,https://www.redfin.com/FL/Delray-Beach/400-Sea...,Scott Porter,John P. O'Grady LLC,None,None,Charles Lynn,One Sotheby's International Realty,None,None,7,blue,Seagate Manor Condo at 400 Seasage Dr #204 in ...
7,PAST SALE,2025-02-12,Condo/Co-op,99 Tresana Blvd #5,Jupiter,FL,33478.0,885000.0,2.0,2.0,Jupiter Country Club,2124.0,NaN,2013.0,NaN,417.0,1293.0,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/99-

In [56]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}\n")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}\n")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}\n")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}\n")


print("\n--------SALE LINKS FOR REFERENCE ONLY--------\n")
print('Top sale: ',df_top_ten['URL'].iloc[0])
print('Second priciest sale: ', df_top_ten['URL'].iloc[1])
print('Cheapest top ten sale: ',df_top_ten['URL'].iloc[-1])
print('Second Cheapest top ten sale: ',df_top_ten['URL'].iloc[-2])
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print('Highest top ten psf: ',max_url)

# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print('Lowest top ten psf: ', min_url)


HED: Alina Residences closing tops Palm Beach County weekly condo sales 
DEK: Top 10 sales ranged from $810,000 to $3,190,000
FEATURED HED:
SEO HED: Palm Beach County County Weekly Condo Report 
SEO DESCRIPTION: Palm Beach County County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Palm Beach County condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Palm Beach County, weekly condo sales, Boca Raton, Jupiter, Highland Beach, Delray Beach

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Palm Beach County County’s NEWS PEG HERE.

Brokers closed 87 condo sales totaling $33,320,130 million from Feb. 10th to Feb. 17th. The previous week, brokers closed 70 condo sales totaling $61,300,000.

Last week’s units sold for an average of $382,990, lower than the $750,000 average pric

In [64]:
print('Top sale:', df_top_ten['URL'].iloc[0])

Top sale: https://www.redfin.com/FL/Boca-Raton/200-SE-Mizner-Blvd-33432/unit-203/home/175170974


In [65]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Boca-Raton/300-SE-5th-Ave-33432/unit-7120/home/42325567


In [66]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Delray-Beach/318-SE-7th-Ave-33483/unit-3/home/42581527


In [67]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Delray-Beach/2000-S-Ocean-Blvd-33483/unit-707/home/42667895


In [68]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Boca-Raton/200-SE-Mizner-Blvd-33432/unit-203/home/175170974


In [69]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Jupiter/99-Tresana-Blvd-33478/unit-5/home/45429760


In [63]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-02-12,Condo/Co-op,200 SE Mizner Blvd #203,Boca Raton,FL,33432.0,3190000.0,2.0,3.5,Alina Residences,2713.0,NaN,2020.0,NaN,1176.0,3329.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/200-SE-Mi...,Beaches MLS,RX-11041137,N,Y,26.347715,-80.083263,https://www.redfin.com/FL/Boca-Raton/200-SE-Mi...,Senada Adzem,Douglas Elliman,Denver Bright,"Compass Florida, LLC",Aaron Buchbinder,Compass Florida LLC,Steven Seigel,The Agency Florida LLC,1,orange,Alina Residences at 200 SE Mizner Blvd #203 in...
1,PAST SALE,2025-02-14,Condo/Co-op,300 SE 5th Ave #7120,Boca Raton,FL,33432.0,1800000.0,3.0,4.5,Mizner Tower,2465.0,NaN,1989.0,NaN,730.0,3485.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/300-SE-5t...,Beaches MLS,RX-11023565,N,Y,26.346087,-80.076579,https://www.redfin.com/FL/Boca-Raton/300-SE-5t...,Sara Brown,Coldwell Banker/BR,None,None,Adam Levitt,The Corcoran Group,None,None,2,blue,Mizner Tower at 300 SE 5th Ave #7120 in Boca R...
2,PAST SALE,2025-02-14,Condo/Co-op,4740 S Ocean Blvd #1610,Highland Beach,FL,33487.0,1500000.0,3.0,2.5,Braemar,2070.0,NaN,1982.0,NaN,725.0,2144.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/4740-...,Beaches MLS,RX-10899388,N,Y,26.393421,-80.069121,https://www.redfin.com/FL/Highland-Beach/4740-...,Gina Finsilver,Compass Florida LLC,None,None,Carol Rochon,"United Realty Group, Inc",None,None,3,blue,Braemar at 4740 S Ocean Blvd #1610 in Highland...
3,PAST SALE,2025-02-14,Condo/Co-op,4161 S Us Highway 1 Unit K4,Jupiter,FL,33477.0,1400000.0,4.0,3.0,Beachcomber Condo,2370.0,NaN,1984.0,NaN,591.0,1595.0,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/4161-S-US-Hi...,Beaches MLS,RX-11025987,N,Y,26.894166,-80.058991,https://www.redfin.com/FL/Jupiter/4161-S-US-Hi...,Andrew Russo,Illustrated Properties LLC (Jupiter),Milla Russo,Illustrated Properties LLC (Jupiter),Mary Ellen Kavanewsky,"NV Realty Group, LLC",None,None,4,blue,Beachcomber Condo at 4161 S Us Highway 1 Unit ...
4,PAST SALE,2025-02-12,Condo/Co-op,500 S Ocean Blvd #1103,Boca Raton,FL,33432.0,1370000.0,2.0,2.0,Chalfonte Condo,1446.0,NaN,1974.0,NaN,947.0,1525.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/500-S-Oce...,Beaches MLS,RX-11034349,N,Y,26.345068,-80.071816,https://www.redfin.com/FL/Boca-Raton/500-S-Oce...,Ana Londono,Lang Realty/BR,None,None,Kun Li,Rainbow Investment Realty Inc,None,None,5,blue,Chalfonte Condo at 500 S Ocean Blvd #1103 in B...
5,PAST SALE,2025-02-14,Condo/Co-op,2800 S Ocean Blvd Unit 10-K,Boca Raton,FL,33432.0,1160000.0,2.0,2.0,Ocean Towers Condominium,1423.0,NaN,1976.0,NaN,815.0,1262.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/2800-S-Oc...,Beaches MLS,RX-11008547,N,Y,26.323476,-80.075019,https://www.redfin.com/FL/Boca-Raton/2800-S-Oc...,Richard Dusik,Florida Beach Realty,None,None,Richard Dusik,Florida Beach Realty,None,None,6,blue,Ocean Towers Condominium at 2800 S Ocean Blvd ...
6,PAST SALE,2025-02-14,Condo/Co-op,400 Seasage Dr #204,Delray Beach,FL,33483.0,1075000.0,2.0,2.0,Seagate Manor Condo,1760.0,NaN,1969.0,NaN,611.0,1504.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/400-Sea...,Beaches MLS,RX-11034650,N,Y,26.454271,-80.063637,https://www.redfin.com/FL/Delray-Beach/400-Sea...,Scott Porter,John P. O'Grady LLC,None,None,Charles Lynn,One Sotheby's International Realty,None,None,7,blue,Seagate Manor Condo at 400 Seasage Dr #204 in ...
7,PAST SALE,2025-02-12,Condo/Co-op,99 Tresana Blvd #5,Jupiter,FL,33478.0,885000.0,2.0,2.0,Jupiter Country Club,2124.0,NaN,2013.0,NaN,417.0,1293.0,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/99-

## Time on Market Calculator

In [84]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-02-12,Condo/Co-op,200 SE Mizner Blvd #203,Boca Raton,FL,33432.0,3190000.0,2.0,3.5,Alina Residences,2713.0,NaN,2020.0,NaN,1176.0,3329.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/200-SE-Mi...,Beaches MLS,RX-11041137,N,Y,26.347715,-80.083263,https://www.redfin.com/FL/Boca-Raton/200-SE-Mi...,Senada Adzem,Douglas Elliman,Denver Bright,"Compass Florida, LLC",Aaron Buchbinder,Compass Florida LLC,Steven Seigel,The Agency Florida LLC,1,orange,Alina Residences at 200 SE Mizner Blvd #203 in...
1,PAST SALE,2025-02-14,Condo/Co-op,300 SE 5th Ave #7120,Boca Raton,FL,33432.0,1800000.0,3.0,4.5,Mizner Tower,2465.0,NaN,1989.0,NaN,730.0,3485.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/300-SE-5t...,Beaches MLS,RX-11023565,N,Y,26.346087,-80.076579,https://www.redfin.com/FL/Boca-Raton/300-SE-5t...,Sara Brown,Coldwell Banker/BR,None,None,Adam Levitt,The Corcoran Group,None,None,2,blue,Mizner Tower at 300 SE 5th Ave #7120 in Boca R...
2,PAST SALE,2025-02-14,Condo/Co-op,4740 S Ocean Blvd #1610,Highland Beach,FL,33487.0,1500000.0,3.0,2.5,Braemar,2070.0,NaN,1982.0,NaN,725.0,2144.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/4740-...,Beaches MLS,RX-10899388,N,Y,26.393421,-80.069121,https://www.redfin.com/FL/Highland-Beach/4740-...,Gina Finsilver,Compass Florida LLC,None,None,Carol Rochon,"United Realty Group, Inc",None,None,3,blue,Braemar at 4740 S Ocean Blvd #1610 in Highland...
3,PAST SALE,2025-02-14,Condo/Co-op,4161 S Us Highway 1 Unit K4,Jupiter,FL,33477.0,1400000.0,4.0,3.0,Beachcomber Condo,2370.0,NaN,1984.0,NaN,591.0,1595.0,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/4161-S-US-Hi...,Beaches MLS,RX-11025987,N,Y,26.894166,-80.058991,https://www.redfin.com/FL/Jupiter/4161-S-US-Hi...,Andrew Russo,Illustrated Properties LLC (Jupiter),Milla Russo,Illustrated Properties LLC (Jupiter),Mary Ellen Kavanewsky,"NV Realty Group, LLC",None,None,4,blue,Beachcomber Condo at 4161 S Us Highway 1 Unit ...
4,PAST SALE,2025-02-12,Condo/Co-op,500 S Ocean Blvd #1103,Boca Raton,FL,33432.0,1370000.0,2.0,2.0,Chalfonte Condo,1446.0,NaN,1974.0,NaN,947.0,1525.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/500-S-Oce...,Beaches MLS,RX-11034349,N,Y,26.345068,-80.071816,https://www.redfin.com/FL/Boca-Raton/500-S-Oce...,Ana Londono,Lang Realty/BR,None,None,Kun Li,Rainbow Investment Realty Inc,None,None,5,blue,Chalfonte Condo at 500 S Ocean Blvd #1103 in B...
5,PAST SALE,2025-02-14,Condo/Co-op,2800 S Ocean Blvd Unit 10-K,Boca Raton,FL,33432.0,1160000.0,2.0,2.0,Ocean Towers Condominium,1423.0,NaN,1976.0,NaN,815.0,1262.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/2800-S-Oc...,Beaches MLS,RX-11008547,N,Y,26.323476,-80.075019,https://www.redfin.com/FL/Boca-Raton/2800-S-Oc...,Richard Dusik,Florida Beach Realty,None,None,Richard Dusik,Florida Beach Realty,None,None,6,blue,Ocean Towers Condominium at 2800 S Ocean Blvd ...
6,PAST SALE,2025-02-14,Condo/Co-op,400 Seasage Dr #204,Delray Beach,FL,33483.0,1075000.0,2.0,2.0,Seagate Manor Condo,1760.0,NaN,1969.0,NaN,611.0,1504.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/400-Sea...,Beaches MLS,RX-11034650,N,Y,26.454271,-80.063637,https://www.redfin.com/FL/Delray-Beach/400-Sea...,Scott Porter,John P. O'Grady LLC,None,None,Charles Lynn,One Sotheby's International Realty,None,None,7,blue,Seagate Manor Condo at 400 Seasage Dr #204 in ...
7,PAST SALE,2025-02-12,Condo/Co-op,99 Tresana Blvd #5,Jupiter,FL,33478.0,885000.0,2.0,2.0,Jupiter Country Club,2124.0,NaN,2013.0,NaN,417.0,1293.0,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/99-

In [83]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2024, 9, 25) ## List (Earlier) date
date2 = datetime(2025, 2, 14) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

142


## Clean CSV for Datawrapper Chart

In [72]:
chart_df = df_top_ten

In [73]:
chart_df = chart_df.fillna(" ")

In [74]:
chart_df['LISTING INFO'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING INFO'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [75]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [76]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "LOCATION"
]

In [77]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [78]:
chart_df['$/SQUARE FEET'].astype(float)

0    1176.0
1     730.0
2     725.0
3     591.0
4     947.0
5     815.0
6     611.0
7     417.0
8     486.0
9     585.0
Name: $/SQUARE FEET, dtype: float64

In [79]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['$/SQUARE FEET'] = chart_df['$/SQUARE FEET'].apply(lambda x: "${:,.0f}".format(x) if pd.notnull(x) else x)
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))
chart_df['BEDS'] = chart_df['BEDS'].apply(lambda x: "{:.0f}".format(x))

In [80]:
csv_date_string = today.strftime("%m_%d_%Y")

In [81]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv")

In [82]:
chart_df

,SOLD DATE,ADDRESS,PRICE,BEDS,BATHS,SQUARE FEET,YEAR BUILT,$/SQUARE FEET,SOURCE,MLS#,LISTING INFO,BUYING INFO
0,2025-02-12,200 SE Mizner Blvd #203 in Boca Raton,"$3,190,000",2,3.5,"2,713",2020,"$1,176",Beaches MLS,RX-11041137,Senada Adzem Douglas Elliman Denver Bright Com...,Aaron Buchbinder Compass Florida LLC Steven Se...
1,2025-02-14,300 SE 5th Ave #7120 in Boca Raton,"$1,800,000",3,4.5,"2,465",1989,$730,Beaches MLS,RX-11023565,Sara Brown Coldwell Banker/BR,Adam Levitt The Corcoran Group
2,2025-02-14,4740 S Ocean Blvd #1610 in Highland Beach,"$1,500,000",3,2.5,"2,070",1982,$725,Beaches MLS,RX-10899388,Gina Finsilver Compass Florida LLC,"Carol Rochon United Realty Group, Inc"
3,2025-02-14,4161 S Us Highway 1 Unit K4 in Jupiter,"$1,400,000",4,3.0,"2,370",1984,$591,Beaches MLS,RX-11025987,Andrew Russo Illustrated Properties LLC (Jupit...,"Mary Ellen Kavanewsky NV Realty Group, LLC"
4,2025-02-12,500 S Ocean Blvd #1103 in Boca Raton,"$1,370,000",2,2.0,"1,446",1974,$947,Beaches MLS,RX-11034349,Ana Londono Lang Realty/BR,Kun Li Rainbow Investment Realty Inc
5,2025-02-14,2800 S Ocean Blvd Unit 10-K in Boca Raton,"$1,160,000",2,2.0,"1,423",1976,$815,Beaches MLS,RX-11008547,Richard Dusik Florida Beach Realty,Richard Dusik Florida Beach Realty
6,2025-02-14,400 Seasage Dr #204 in Delray Beach,"$1,075,000",2,2.0,"1,760",1969,$611,Beaches MLS,RX-11034650,Scott Porter John P. O'Grady LLC,Charles Lynn One Sotheby's International Realt...
7,2025-02-12,99 Tresana Blvd #5 in Jupiter,"$885,000",2,2.0,"2,124",2013,$417,Beaches MLS,RX-11041325,Sam Elias The Agency Florida LLC Rosemary Elia...,"Diane Gray NV Realty Group, LLC"
8,2025-02-14,2000 S Ocean Blvd #707 in Delray Beach,"$850,000",2,2.0,"1,750",1973,$486,Beaches MLS,RX-10993954,Aixa Diaz-Monaco Coldwell Banker Realty /Delra...,Lisa Nanfra Compass Florida LLC
9,2025-02-13,318 SE 7th Ave #3 in Delray Beach,"$810,000",2,2.0,"1,385",1978,$585,Beaches MLS,RX-11017285,William Schloesser Houseworks Real Estate Caro...,Carol Fredericks Houseworks Real Estate Willia...
